In [2]:
import torch
torch.cuda.is_available()

True

In [9]:
from ultralytics import YOLO

# 1. Load the model (YOLO11n is the fastest)
model = YOLO("yolo26l.pt")

# 2. Run tracking on your video
# 'show=True' opens a window to watch live
# 'save=True' saves the result to runs/detect/track/
results = model.track(
    source=r"AICity22_Track1_MTMC_Tracking/test/S06/c041/vdo.avi",
    imgsz=1080, conf=0.25,
    classes=[2, 3, 5, 7],
    show=True,
    save=True,
    tracker="botsort.yaml"
)




WARNING imgsz=[1080] must be multiple of max stride 32, updating to [1088]
WARNING 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2001) C:\ComputerVision\car_multicamera\AICity22_Track1_MTMC_Tracking\test\S06\c041\vdo.avi: 832x1088 20 cars, 21.8ms
video 1/1 (frame 2/2001) C:\ComputerVision\car_multicamera\AICity22_Track1_MTMC_Tracking\test\S06\c041\vdo.avi: 832x1088 20 cars, 21.6ms
video 1/1 (frame 3/2001) C:\ComputerVision\car_multicamera\AICity22_Track1_MTMC_Tracking\test\S06\c041\vdo.avi: 

In [3]:
import cv2
from ultralytics import YOLO
import geometry_utils
from collections import defaultdict

# 1. Setup
model = YOLO("yolo11m.pt")
video_path = r"AICity22_Track1_MTMC_Tracking\test\S06\c041/vdo.avi"
cap = cv2.VideoCapture(video_path)

class_names = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}

trajectories = geometry_utils.TrajectoryManager()
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    # 2. Process Frame
    results = model.track(frame, persist=True, classes=[2, 3, 5, 7], imgsz=1088, verbose=False)

    # 3. Draw Manually
    if results[0].boxes is not None and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

        for box, obj_id, cls_id in zip(boxes, ids, class_ids):
            x1, y1, x2, y2 = map(int, box)
            label = f"{class_names.get(cls_id, 'Veh')}: #{obj_id}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            trajectories.update(obj_id, (cx,cy))

        trajectories.draw(frame, ids.tolist())


    # 4. Display
    cv2.imshow("YOLO Step-by-Step", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

    if key == ord(' '):
        do_exit = False
        while True:
            key2 = cv2.waitKey(0) & 0xFF
            if key2 == ord(' '):
                break
            if key2 == ord('q'):
                do_exit = True
                break
        if do_exit:
            break

cap.release()
cv2.destroyAllWindows()